In [1]:
import sys
import torch
import numpy as np
import pandas as ps
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn import metrics


sys.path.append("..")
device = torch.device("cuda:0")

In [2]:
from src.models import MulticlassEfficientNet
from src.datasets import ImagesDataset
from src.metrics import _alaska_weighted_auc
from src.experiment import VALID_AUGMENTATIONS

/home/dmdr/anaconda3/envs/torch/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/home/dmdr/anaconda3/envs/torch/lib/python3.7/site-packages/nbformat/notebooknode.py:4: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working

/home/dmdr/anaconda3/envs/torch/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/home/dmdr/anaconda3/envs/torch/lib/python3.7/site-packages/torchvision/extension.py:11: ResourceWarning:

unclosed file <_io.BufferedReader name='/home/dmdr/anaconda3/envs/torch/lib/python3.7/site-packages/torchvision/_C.so'>



In [3]:
model = MulticlassEfficientNet("efficientnet-b3", 4)
model.load_state_dict(
    torch.load(
        "../logs/efficientnet-b3-c4-grouped7/checkpoints/best.pth",
#         "../logs/efficientnet-b0-magic-constant-continue2/checkpoints/best.pth", 
        map_location="cpu"
    )["model_state_dict"]
)
model = model.to(device)
model = model.eval()

Loaded pretrained weights for efficientnet-b3


In [4]:
data = ps.read_csv("../data/combined_group.csv")
data = data[data["folds"] == 0]
data["images"] = "../" + data["images"]

print(data.shape)
print(data["labels"].value_counts())
data.head()

(60000, 4)
1    45000
0    15000
Name: labels, dtype: int64


,images,labels,classes,folds
0,../data/resized_data/Cover/28292.jpg,0,0,0
2,../data/resized_data/Cover/75472.jpg,0,0,0
3,../data/resized_data/Cover/32819.jpg,0,0,0
5,../data/resized_data/Cover/42993.jpg,0,0,0
6,../data/resized_data/Cover/70794.jpg,0,0,0


In [5]:
dataset = ImagesDataset(data["images"].values, transforms=VALID_AUGMENTATIONS)
dataloader = DataLoader(dataset, batch_size=64, num_workers=16)

In [6]:
preds = []
with torch.no_grad():
    for batch in tqdm(dataloader):
        batch = batch.to(device)
        out = 1 - F.softmax(model(batch), dim=1).data.cpu().numpy()[:,0]
        preds.append(out)

preds = np.concatenate(preds)
print(preds.shape)

100%|██████████| 938/938 [14:11<00:00,  1.10it/s]

(60000,)


In [24]:
labels = data["labels"].values
_preds = preds

auc = metrics.roc_auc_score(labels, _preds)
weighted_auc = _alaska_weighted_auc(labels, _preds)
accuracy = (labels == (_preds >= 0.5)).mean()


print(f'   Accuracy = {accuracy}')
print(f'        AUC = {auc}')
print(f'WeightedAUC = {weighted_auc}')

   Accuracy = 0.8259
        AUC = 0.8806701222222222
WeightedAUC = 0.9145383126984128


In [28]:
labels = data["labels"].values

_preds = preds ** 2

auc = metrics.roc_auc_score(labels, _preds)
weighted_auc = _alaska_weighted_auc(labels, _preds)
accuracy = (labels == (_preds >= 0.5)).mean()


print(f'   Accuracy = {accuracy}')
print(f'        AUC = {auc}')
print(f'WeightedAUC = {weighted_auc}')

   Accuracy = 0.75205
        AUC = 0.8806701222222222
WeightedAUC = 0.9145383126984128


In [20]:
labels = data["labels"].values

_preds = preds ** 4

auc = metrics.roc_auc_score(labels, _preds)
weighted_auc = _alaska_weighted_auc(labels, _preds)
accuracy = (labels == (_preds >= 0.5)).mean()


print(f'   Accuracy = {accuracy}')
print(f'        AUC = {auc}')
print(f'WeightedAUC = {weighted_auc}')

   Accuracy = 0.6919833333333333
        AUC = 0.8806701222222222
WeightedAUC = 0.9145383126984128


In [19]:
labels = data["labels"].values

_preds = preds ** 8

auc = metrics.roc_auc_score(labels, _preds)
weighted_auc = _alaska_weighted_auc(labels, _preds)
accuracy = (labels == (_preds >= 0.5)).mean()


print(f'   Accuracy = {accuracy}')
print(f'        AUC = {auc}')
print(f'WeightedAUC = {weighted_auc}')

   Accuracy = 0.6688666666666667
        AUC = 0.8806701222222222
WeightedAUC = 0.9145383126984128


## another checkpoint

In [29]:
model = MulticlassEfficientNet("efficientnet-b3", 4)
model.load_state_dict(
    torch.load(
        "../logs/efficientnet-b3-c4-grouped6/checkpoints/best.pth",
#         "../logs/efficientnet-b0-magic-constant-continue2/checkpoints/best.pth", 
        map_location="cpu"
    )["model_state_dict"]
)
model = model.to(device)
model = model.eval()

Loaded pretrained weights for efficientnet-b3


In [30]:
preds2 = []
with torch.no_grad():
    for batch in tqdm(dataloader):
        batch = batch.to(device)
        out = 1 - F.softmax(model(batch), dim=1).data.cpu().numpy()[:,0]
        preds2.append(out)

preds2 = np.concatenate(preds2)
print(preds2.shape)

100%|██████████| 938/938 [13:37<00:00,  1.15it/s]

(60000,)


In [34]:
labels = data["labels"].values
_preds = preds

auc = metrics.roc_auc_score(labels, _preds)
weighted_auc = _alaska_weighted_auc(labels, _preds)
accuracy = (labels == (_preds >= 0.5)).mean()


print(f'   Accuracy = {accuracy}')
print(f'        AUC = {auc}')
print(f'WeightedAUC = {weighted_auc}')

   Accuracy = 0.8259
        AUC = 0.8806701222222222
WeightedAUC = 0.9145383126984128


In [35]:
labels = data["labels"].values
_preds = preds2

auc = metrics.roc_auc_score(labels, _preds)
weighted_auc = _alaska_weighted_auc(labels, _preds)
accuracy = (labels == (_preds >= 0.5)).mean()


print(f'   Accuracy = {accuracy}')
print(f'        AUC = {auc}')
print(f'WeightedAUC = {weighted_auc}')

   Accuracy = 0.8248333333333333
        AUC = 0.8795510903703704
WeightedAUC = 0.913848753968254


## test power blend

In [41]:
labels = data["labels"].values
_preds = (preds + preds2) / 2

auc = metrics.roc_auc_score(labels, _preds)
weighted_auc = _alaska_weighted_auc(labels, _preds)
accuracy = (labels == (_preds >= 0.5)).mean()


print(f'   Accuracy = {accuracy}')
print(f'        AUC = {auc}')
print(f'WeightedAUC = {weighted_auc}')

   Accuracy = 0.8260666666666666
        AUC = 0.8807713881481481
WeightedAUC = 0.9146921444444446


In [42]:
labels = data["labels"].values
_preds = (preds ** 2 + preds2 ** 2) / 2

auc = metrics.roc_auc_score(labels, _preds)
weighted_auc = _alaska_weighted_auc(labels, _preds)
accuracy = (labels == (_preds >= 0.25)).mean()


print(f'   Accuracy = {accuracy}')
print(f'        AUC = {auc}')
print(f'WeightedAUC = {weighted_auc}')

   Accuracy = 0.82605
        AUC = 0.8807900577777777
WeightedAUC = 0.9147053989417989


In [39]:
labels = data["labels"].values
_preds = (preds ** 4 + preds2 ** 4) / 2

auc = metrics.roc_auc_score(labels, _preds)
weighted_auc = _alaska_weighted_auc(labels, _preds)
accuracy = (labels == (_preds >= 0.5)).mean()


print(f'   Accuracy = {accuracy}')
print(f'        AUC = {auc}')
print(f'WeightedAUC = {weighted_auc}')

   Accuracy = 0.68685
        AUC = 0.8807708488888888
WeightedAUC = 0.9146915682539684


In [40]:
labels = data["labels"].values
_preds = (preds ** 8 + preds2 ** 8) / 2

auc = metrics.roc_auc_score(labels, _preds)
weighted_auc = _alaska_weighted_auc(labels, _preds)
accuracy = (labels == (_preds >= 0.5)).mean()


print(f'   Accuracy = {accuracy}')
print(f'        AUC = {auc}')
print(f'WeightedAUC = {weighted_auc}')

   Accuracy = 0.662
        AUC = 0.880747277037037
WeightedAUC = 0.9146744814814814


## score on all train data (train + validation)

**NOTE:** this score will be different from score on validation

In [7]:
labels = data["labels"].values

auc = metrics.roc_auc_score(labels, preds)
weighted_auc = _alaska_weighted_auc(labels, preds)
accuracy = (labels == (preds >= 0.5)).mean()


print(f'   Accuracy = {round(accuracy, 5)}')
print(f'        AUC = {round(auc, 5)}')
print(f'WeightedAUC = {round(weighted_auc, 5)}')

   Accuracy = 0.62635
        AUC = 0.77973
WeightedAUC = 0.84226


In [8]:
print(metrics.classification_report(labels, preds >= 0.5))

              precision    recall  f1-score   support

           0       0.39      0.85      0.53     15000
           1       0.92      0.55      0.69     45000

    accuracy                           0.63     60000
   macro avg       0.65      0.70      0.61     60000
weighted avg       0.79      0.63      0.65     60000



In [9]:
pred_classes = (preds >= 0.5)

print("Positive accuracy -", (labels == pred_classes)[labels == 1].mean())
print("Negative accuracy -", (labels == pred_classes)[labels == 0].mean())

Positive accuracy - 0.5510222222222222
Negative accuracy - 0.8523333333333334


## validation score

In [10]:
fold_labels = data[data["folds"] == 0]["labels"].values
fold_preds = preds[data["folds"] == 0]


auc = metrics.roc_auc_score(fold_labels, fold_preds)
weighted_auc = _alaska_weighted_auc(fold_labels, fold_preds)
accuracy = (fold_labels == (fold_preds >= 0.5)).mean()


print(f'   Accuracy = {round(accuracy, 5)}')
print(f'        AUC = {round(auc, 5)}')
print(f'WeightedAUC = {round(weighted_auc, 5)}')

   Accuracy = 0.75417
        AUC = 0.73491
WeightedAUC = 0.80841


In [11]:
print(metrics.classification_report(fold_labels, fold_preds >= 0.5))

              precision    recall  f1-score   support

           0       0.59      0.06      0.10     15000
           1       0.76      0.99      0.86     45000

    accuracy                           0.75     60000
   macro avg       0.67      0.52      0.48     60000
weighted avg       0.72      0.75      0.67     60000



In [12]:
pred_classes = (fold_preds >= 0.5)

print("Positive accuracy -", (fold_labels == pred_classes)[fold_labels == 1].mean())
print("Negative accuracy -", (fold_labels == pred_classes)[fold_labels == 0].mean())

Positive accuracy - 0.9872
Negative accuracy - 0.05506666666666667
